In [8]:
from pennylane import numpy as np
import pennylane as qml
from matplotlib import pyplot as plt

from functions_vqe_crx_3 import vqe_hee, vqe_ng
import numpy as np
from joblib import Parallel, delayed

device = 'cpu'

## Distribution of results

### Parameters

In [30]:
max_iterations = 100

layers = np.arange(2, 31, 4)
n_layers = len(layers)

n_runs = 100

conv_tol = 1e-06

weights_lr = 0.002
phi_lr = 0.05

theta = np.pi

max_workers = -1

In [31]:
n_cells = (2, 2)
coupling = [1,1,1]

qubits = n_cells[0] * n_cells[1]

lattice = 'triangle'

H = qml.spin.heisenberg(lattice, n_cells, coupling, boundary_condition=True)

In [32]:
prefix1 = "../local_data/TAFH_log1/"
spec1 = "_crx_1"

prefix2 = prefix1
spec2 = "_crx_2"

In [33]:
import json

run_info = {
    "max_iterations": max_iterations,
    "layers": layers.tolist(),
    "n_layers": n_layers,
    "n_runs": n_runs,
    "conv_tol": conv_tol,
    "weights_lr": weights_lr,
    "phi_lr": phi_lr,
    "prefix1": prefix1,
    "spec1": spec1,
    "prefix2": prefix2,
    "spec2": spec2,
    "device": device,
    "qubits": qubits,
    "theta": theta,
    "opt": "SDG",
    "ansatz": "double cascade with CRX gates and correlated angles",
}

with open(prefix1 + "run_info.txt", "w") as f:
    json.dump(run_info, f, indent=4)

In [34]:
def run_single_simulation_1(i, l, j):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_hee(H, qubits, l, lr=weights_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)
    
    return i, j, (energy, grad_norm, grad_var, conv)

def run_single_simulation_2(i, l, j):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_ng(H, qubits, l, weights_lr=weights_lr, phi_lr=phi_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)

    return i, j, (energy, grad_norm, grad_var, conv)

### HEE

In [35]:
tasks = (delayed(run_single_simulation_1)(i, l, j)
         for i, l in enumerate(layers)
         for j in range(n_runs))
print('number of tasks:', n_layers * n_runs)
results = Parallel(n_jobs=max_workers, verbose=11)(tasks)

number of tasks: 800


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  

#### Save

In [36]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_layers, n_runs, max_iterations+1))
grad_norms = np.zeros((n_layers, n_runs, max_iterations))
grad_variances = np.zeros((n_layers, n_runs, max_iterations))
convergences = np.zeros((n_layers, n_runs), dtype=bool)

for i, j, result in results:
    energies[i, j, :] = np.array(result[0])
    grad_norms[i, j, :] = np.array(result[1])
    grad_variances[i, j, :] = np.array(result[2])
    convergences[i, j] = result[3]

In [37]:
np.save(prefix1+"energies"+spec1+".npy", energies)
np.save(prefix1+"grad_norms"+spec1+".npy", grad_norms)
np.save(prefix1+"grad_variances"+spec1+".npy", grad_variances)
np.save(prefix1+"convergences"+spec1+".npy",convergences)

### NG

In [38]:
tasks = (delayed(run_single_simulation_2)(i, l, j)
         for i, l in enumerate(layers)
         for j in range(n_runs))
print('number of tasks:', n_layers * n_runs)
results = Parallel(n_jobs=max_workers, verbose=11)(tasks)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.


number of tasks: 800


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.7s
[Paralle

#### Save

In [39]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_layers, n_runs, max_iterations+1))
grad_norms = np.zeros((n_layers, n_runs, max_iterations))
grad_variances = np.zeros((n_layers, n_runs, max_iterations))
convergences = np.zeros((n_layers, n_runs), dtype=bool)

for i, j, result in results:
    energies[i, j, :] = np.array(result[0])
    grad_norms[i, j, :] = np.array(result[1])
    grad_variances[i, j, :] = np.array(result[2])
    convergences[i, j] = result[3]

In [40]:
np.save(prefix2+"energies"+spec2+".npy", energies)
np.save(prefix2+"grad_norms"+spec2+".npy", grad_norms)
np.save(prefix2+"grad_variances"+spec2+".npy", grad_variances)
np.save(prefix2+"convergences"+spec2+".npy",convergences)